In [54]:
import sys, time, math, copy
import numpy as np
import gym
from gym import spaces, logger
from gym.utils import seeding

from robo_gym.utils.exceptions import InvalidStateError, RobotServerError
import robo_gym_server_modules.robot_server.client as rs_client
from robo_gym.envs.simulation_wrapper import Simulation
from robo_gym_server_modules.robot_server.grpc_msgs.python import robot_server_pb2

from typing import List

In [59]:
class Mir100Env(gym.Env):
    real_robot = False
    map_size = 256
    resolution = 0.05
    
    def __init__(self, rs_address=None, max_episode_steps=500, **kwargs):
        self.mir100 = mir100_util.Mir100()
        self.max_episde_steps = max_episode_steps
        self.elapsed_steps = 0
        self.observation_space = self._get_observation_space()
        half = self.map_size*self.resolution/2

        self.action_space = spaces.Dict({
            'position': spaces.Box(low=-half, high=half, shape=(2,), dtype=np.float32),
            'orientation': spaces.Box(low=0, high=np.pi*2, dtype=np.float32)
        })
        self.seed()
        self.distance_threshold = 0.2
        self.min_target_dist = 1.0
        
        # Connect to Robot Server
        if rs_address:
            self.client = rs_client.Client(rs_address)
        else:
            print("WARNING: No IP and Port passed. Simulation will not be started")
            print("WARNING: Use this only to get environment shape")
            
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
    
    def reset(self, 
              obstacle_count: int=10, 
              obstacle_size: float=0.7, 
              target_size: float=0.2, 
              room_length_max: float=9.0, 
              room_mass_min: float=20.0, 
              room_mass_max: float=36.0, 
              room_wall_height: float=0.8, 
              room_wall_thickness: float=0.05,
              target_poses:List[List[float]]=None):
        
        self.elapsed_steps = 0
        self.prev_base_reward = None
        
        # Initialize environment state
        self.state = np.zeros(self._get_env_state_len())
        rs_state = np.zeros(self._get_robot_server_state_len())
        
        # Set Robot starting position
        
        

In [44]:
gym.spaces.Box(low=-10,high=10, shape=(2,))

Box(-10.0, 10.0, (2,), float32)